In [39]:
%pip install -q pyomo
!apt-get install -y -qq coinor-cbc

Note: you may need to restart the kernel to use updated packages.
E: Could not open lock file /var/lib/dpkg/lock-frontend - open (13: Permission denied)
E: Unable to acquire the dpkg frontend lock (/var/lib/dpkg/lock-frontend), are you root?


In [51]:
from pyomo.environ import *
import math
n=5
b=100
M=5000  #Big Number. This is the only thing i am not able to make sure what to select. If i select too big, it gives 400, which i dont think is possible
model=ConcreteModel()

In [41]:
model.xi = Var(range(n),domain= NonNegativeIntegers)
model.ri= Var(range(n),domain= NonNegativeIntegers)
model.y=Var(range(n),domain=Binary)

In [42]:
model.obj = Objective(expr = b-sum(model.xi[i] for i in range(n))+sum(model.ri[i] for i in range(n)), sense = maximize)

In [43]:
model.constraints = ConstraintList()
model.constraints.add(expr = model.ri[0] == 0)                              #Initially for first 2 days, no returns
model.constraints.add(expr = model.ri[1] == 0)
for i in range(n):
    model.constraints.add(expr = model.xi[i]>=0)                            #Investments must be poisitive
    model.constraints.add(expr = model.ri[i]>=0)                            #Returns must be positive
    model.constraints.add(expr = b-sum(model.xi[j] for j in range(i+1))+sum(model.ri[j] for j in range(i+1))>=0) #Money always greater than 0

In [44]:
for i in [2,3,4]:
  model.constraints.add(expr = model.xi[i-1]-2*model.xi[i-2]<=M*(1-model.y[i]))     #BigM method used
  model.constraints.add(expr = model.xi[i-1]-2*model.xi[i-2]>=-M*(1-model.y[i]))    #Makes sure y[i] is 0 if x[i-1]!=2x[i-2]
  ###########################################
  #But how to make sure y[i] is 1 otherwise?
  ###########################################
  model.constraints.add(expr = model.ri[i]<=M*(model.y[i]))                       #Makes sure r[i] is 0 if y[i] is 0
  model.constraints.add(expr = model.ri[i]>=-M*(model.y[i]))
  model.constraints.add(expr = model.ri[i]-2*model.xi[i-2]<=M*(1-model.y[i]))         #Makes sure r[i] is 2x[i-2] if y[i] is 1
  model.constraints.add(expr = model.ri[i]-2*model.xi[i-2]>=-M*(1-model.y[i]))

In [45]:
opt = SolverFactory('cbc')

In [46]:
result = opt.solve(model, tee = True)
print('Solver status:', result.solver.status)
print('Solver termination condition:',result.solver.termination_condition)

Welcome to the CBC MILP Solver 
Version: 2.10.8 
Build Date: May  9 2022 

command line - /bin/cbc -printingOptions all -import /tmp/tmpzwnnrusb.pyomo.lp -stat=1 -solve -solu /tmp/tmpzwnnrusb.pyomo.soln (default strategy 1)
Option for printingOptions changed from normal to all
 CoinLpIO::readLp(): Maximization problem reformulated as minimization
Coin0009I Switching back to maximization to get correct duals etc
Presolve is modifying 1 integer bounds and re-presolving
Presolve 19 (-16) rows, 11 (-3) columns and 62 (-28) elements
Statistics for presolved model
Original problem has 13 integers (3 of which binary)
Presolved problem has 11 integers (3 of which binary)
==== 3 zero objective 3 different
5 variables have objective of -1
3 variables have objective of -0
3 variables have objective of 1
==== absolute objective values 2 different
3 variables have objective of 0
8 variables have objective of 1
==== for integers 3 zero objective 3 different
5 variables have objective of -1
3 variabl

In [47]:
print('Optimal money', model.obj())

Optimal money 100.0
